# Trabajo integrador - Parte 1
## Python y Numpy

**Nombre**:

In [44]:
import numpy as np

## Ejercicio 1

Dada una matriz en formato *numpy array*, donde cada fila de la matriz representa un vector matemático, se requiere computar las normas $l_0$, $l_1$, $l_2$, $l_{\infty}$, según la siguientes definiciones:

\begin{equation}
    ||\mathbf{x}||^{p} = \bigg(\sum_{j=1}^{n}{|x_i|^p}\bigg)^{\frac{1}{p}}
\end{equation}

con los casos especiales para $p=0$ y $p=\infty$ siendo:

\begin{equation}
    \begin{array}{rcl}
        ||\mathbf{x}||_0 & = & \bigg(\sum_{j=1 \wedge x_j != 0}{|x_i|}\bigg)\\
        ||\mathbf{x}||_{\infty} & = & \max_{i}{|x_i|}\\
    \end{array}
\end{equation}

In [45]:
#DEFINICION DE FUNCIONES
# Procesamiento de la norma en la fila
def processRowNorm(r: np.ndarray, n: float):
    if n == 0:
        return np.count_nonzero(r)
    elif np.isinf(n):
        return np.max(np.abs(r))
    else:
        return np.sum(np.abs(r) ** n) ** (1 / n)

# Procesamiento de la norma en la matriz
def norm(m: np.ndarray, n: float):
    return np.apply_along_axis(processRowNorm, 1, m, n)

In [46]:
# EJECUCION DE FUNCIONES
m = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [0, 10, 0]])

print(norm(m, 0))
print(norm(m, np.inf))
print(norm(m, 2))

[3 3 3 1]
[ 3  6  9 10]
[ 3.74165739  8.77496439 13.92838828 10.        ]


## Ejercicio 2

En clasificación contamos con dos arreglos, la “verdad” y la “predicción”. Cada elemento de los arreglos pueden tomar dos valores, “True” (representado por 1) y “False” (representado por 0). Entonces podemos definir 4 variables:

* True Positive (TP): El valor verdadero es 1 y el valor predicho es 1
* True Negative (TN): El valor verdadero es 0 y el valor predicho es 0
* False Positive (FP): El valor verdadero es 0 y el valor predicho es 1
* False Negative (FN): El valor verdadero es 1 y el valor predicho es 0

A partir de esto definimos:

* Precision = TP / (TP + FP)
* Recall = TP / (TP + FN)
* Accuracy = (TP + TN) / (TP + TN + FP + FN)
 
Calcular las 3 métricas con Numpy y operaciones vectorizadas.

In [47]:
# DEFINICION DE FUNCION
def hypothesisMetrics(prediction: np.ndarray, truth: np.ndarray):
    TP = (prediction & truth).astype(int).sum()
    TN = np.logical_not(prediction | truth).astype(int).sum()
    FP = (prediction & np.logical_not(truth)).astype(int).sum()
    FN = np.logical_not(prediction | np.logical_not(truth)).astype(int).sum()

    return {
        "precision": TP / (TP + FP),
        "recall": TP / (TP + FN),
        "accuracy": (TP + TN) / (TP + TN + FP + FN),
    }

In [48]:
#EJECUCION DE FUNCION
prediction = np.array([1,1,1,1,0,0,1,1,0,0])
truth = np.array     ([1,1,0,1,1,1,0,0,0,1])

print(hypothesisMetrics(prediction, truth))

{'precision': 0.5, 'recall': 0.5, 'accuracy': 0.4}


## Ejercicio 3

Crear una función que separe los datos en train-validation-test. Debe recibir de parametros:

- X: Array o Dataframe que contiene los datos de entrada del sistema.
- y: Array o Dataframe que contiene la(s) variable(s) target del problema.
- train_percentage: _float_ el porcentaje de training.
- test_percentage: _float_ el porcentaje de testing.
- val_percentage: _float_ el porcentaje de validación.
- shuffle: _bool_ determina si el split debe hacerse de manera random o no.

Hints: 

* Usar Indexing y slicing
* Usar np.random.[...]

In [62]:
# DEFINICION DE FUNCION
def split(
    X_input: np.ndarray,
    Y_input: np.ndarray,
    train_size: float = 0.7,
    val_size: float = 0.15,
    test_size: float = 0.15,
    random_state: int = 42,
    shuffle: bool = True,
):
    dataset_size = len(X_input)
    indexes = np.array(range(dataset_size))

    if shuffle:
        if random_state is not None:
            np.random.seed(random_state)
        np.random.shuffle(indexes)

    indexFrom = indexTo = 0
    train_indexes = val_indexes = test_indexes = np.array([])

    if train_size > 0:
        indexTo += int(dataset_size * train_size)
        train_indexes = indexes[indexFrom:indexTo]
        indexFrom = indexTo

    if val_size > 0:
        indexTo += int(dataset_size * val_size)
        val_indexes = indexes[indexFrom:indexTo]
        indexFrom = indexTo

    if test_size > 0:
        indexTo += int(dataset_size * test_size)
        test_indexes = indexes[indexFrom:indexTo]

    return {
        "x_train": X_input[train_indexes] if train_size > 0 else [],
        "y_train": Y_input[train_indexes] if train_size > 0 else [],
        "x_val": X_input[val_indexes] if val_size > 0 else [],
        "y_val": Y_input[val_indexes] if val_size > 0 else [],
        "x_test": X_input[test_indexes] if test_size > 0 else [],
        "y_test": Y_input[test_indexes] if test_size > 0 else [],
    }

In [63]:
# EJECUCION DE FUNCIÓN
dataset_X = np.array(
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
)
dataset_Y = np.array(
    [
        1.0,
        2.0,
        3.0,
        4.0,
        5.0,
        6.0,
        7.0,
        8.0,
        9.0,
        10.0,
        11.0,
        12.0,
        13.0,
        14.0,
        15.0,
        16.0,
        17.0,
        18.0,
        19.0,
        20.0,
    ]
)

splitSets = split(dataset_X, dataset_Y, shuffle=False)

print(splitSets)

splitSets = split(dataset_X, dataset_Y, random_state=1024)

print(splitSets)

splitSets = split(dataset_X, dataset_Y, train_size=0.85, val_size=0, shuffle=False)

print(splitSets)

{'x_train': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), 'y_train': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14.]), 'x_val': array([15, 16, 17]), 'y_val': array([15., 16., 17.]), 'x_test': array([18, 19, 20]), 'y_test': array([18., 19., 20.])}
{'x_train': array([11,  9,  5,  7, 17, 15, 12, 16,  8, 20,  3,  4, 14,  1]), 'y_train': array([11.,  9.,  5.,  7., 17., 15., 12., 16.,  8., 20.,  3.,  4., 14.,
        1.]), 'x_val': array([19, 10,  6]), 'y_val': array([19., 10.,  6.]), 'x_test': array([13,  2, 18]), 'y_test': array([13.,  2., 18.])}
{'x_train': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]), 'y_train': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17.]), 'x_val': [], 'y_val': [], 'x_test': array([18, 19, 20]), 'y_test': array([18., 19., 20.])}
